In [1]:
import pandas as pd
import os
import json
import numpy as np
import re
from operator import itemgetter
from glob import glob
import itertools
import csv


## Tokens

In [4]:
strains = "/home/gomez/gomez/preds2/NER_output/STRAIN/805.parquet"

In [5]:
df = pd.read_parquet(strains).explode("ner").dropna()

FileNotFoundError: [Errno 2] No such file or directory: '/home/gomez/gomez/preds2/NER_output/STRAIN/805.parquet'

In [201]:
grouped = df.groupby("text").agg({"ner": lambda x: list(x)}).reset_index()

In [203]:
def merge_entities(entity_list):
    merged_list = []
    skip = False

    for i in range(len(entity_list)):
        if skip:
            skip = False
            continue

        current_entity = entity_list[i]
        current_entity.pop('word', None)  # Remove 'word' entry
        if current_entity['entity_group'] == 'B':
            scores = [current_entity['score']]
            # Look ahead to find contiguous 'I' entities
            next_idx = i + 1
            while next_idx < len(entity_list) and \
                  entity_list[next_idx]['entity_group'] == 'I' and \
                  (entity_list[next_idx]['start'] - current_entity['end']) <= 4:
                scores.append(entity_list[next_idx]['score'])
                current_entity['end'] = entity_list[next_idx]['end']  # Update end value
                skip = True
                next_idx += 1

            # Average the scores
            current_entity['score'] = sum(scores) / len(scores)

        merged_list.append(current_entity)

    return merged_list


In [204]:
grouped['ner'] = grouped['ner'].apply(merge_entities)


In [206]:
grouped = grouped.explode("ner")

In [207]:
grouped

,text,ner
0,"""Coordinate positions or positions of three re...","{'end': 159, 'entity_group': 'B', 'score': 0.6..."
1,"#MW113) were used, and stored in 0.5-0.75 mL o...","{'end': 6, 'entity_group': 'B', 'score': 0.880..."
2,"& Hur (paratype: KBA-L-0001037); South Korea, ...","{'end': 27, 'entity_group': 'B', 'score': 0.78..."
3,"(1) With ectopic NSP1 expression, we observed ...","{'end': 161, 'entity_group': 'B', 'score': 0.7..."
4,(1)Culture Pmet3-CDC20 Pgal-CEN3-tetOs TetR-GF...,"{'end': 66, 'entity_group': 'B', 'score': 0.75..."
...,...,...
9410,经Xho1、EcoR1酶切图筛选Track-Eazyp53(del)、Track-Easy-...,"{'end': 9, 'entity_group': 'B', 'score': 0.658..."
9411,继发T790M突变建议的主要检测手段有cobas、AMRS、ddPCR甚至二代测序（next...,"{'end': 4, 'entity_group': 'B', 'score': 0.632..."
9412,考虑到MPOG-463A在不同种族的差异分布(http://www.ncbi.nlm.nih...,"{'end': 12, 'entity_group': 'B', 'score': 0.78..."
9413,肺癌细胞系A549（CCL#185）购自American Type Culture Coll...,"{'end': 9, 'entity_group': 'B', 'score': 0.773..."


In [208]:
grouped = pd.concat(
                [grouped.drop(columns="ner"), grouped.ner.apply(pd.Series)], axis=1
            )

In [209]:
grouped

,text,end,entity_group,score,start
0,"""Coordinate positions or positions of three re...",159,B,0.652696,153
1,"#MW113) were used, and stored in 0.5-0.75 mL o...",6,B,0.880458,1
2,"& Hur (paratype: KBA-L-0001037); South Korea, ...",27,B,0.789787,17
3,"(1) With ectopic NSP1 expression, we observed ...",161,B,0.705191,158
4,(1)Culture Pmet3-CDC20 Pgal-CEN3-tetOs TetR-GF...,66,B,0.757170,64
...,...,...,...,...,...
9410,经Xho1、EcoR1酶切图筛选Track-Eazyp53(del)、Track-Easy-...,9,B,0.658696,6
9411,继发T790M突变建议的主要检测手段有cobas、AMRS、ddPCR甚至二代测序（next...,4,B,0.632785,2
9412,考虑到MPOG-463A在不同种族的差异分布(http://www.ncbi.nlm.nih...,12,B,0.780138,3
9413,肺癌细胞系A549（CCL#185）购自American Type Culture Coll...,9,B,0.773091,5


In [210]:
grouped['word'] = grouped.apply(lambda row: row['text'][row['start']:row['end']], axis=1)


In [212]:
grouped["word"] = grouped["word"].str.lower()

In [214]:
grouped.to_csv("test.csv")

---

In [4]:
data = "810"
strains = glob(f"/home/gomez/gomez/preds{data}/NER_output/STRAIN/*.parquet")

In [7]:
def merge_entities(entity_list):
    merged_list = []
    skip = False

    for i in range(len(entity_list)):
        if skip:
            skip = False
            continue

        current_entity = entity_list[i]
        current_entity.pop("word", None)  # Remove 'word' entry
        if current_entity["entity_group"] == "B":
            scores = [current_entity["score"]]
            # Look ahead to find contiguous 'I' entities
            next_idx = i + 1
            while (
                next_idx < len(entity_list)
                and entity_list[next_idx]["entity_group"] == "I"
                and (entity_list[next_idx]["start"] - current_entity["end"]) <= 4
            ):
                scores.append(entity_list[next_idx]["score"])
                current_entity["end"] = entity_list[next_idx]["end"]  # Update end value
                skip = True
                next_idx += 1

            # Average the scores
            current_entity["score"] = sum(scores) / len(scores)

        merged_list.append(current_entity)

    return merged_list

In [8]:
l = []
for file in tqdm(strains):
	d = pd.read_parquet(file).explode("ner").dropna()
	grouped = d.groupby("text").agg({"ner": lambda x: list(x)}).reset_index()
	grouped["ner"] = grouped["ner"].apply(merge_entities)
	grouped = grouped.explode("ner")
	grouped = pd.concat(
		[grouped.drop(columns="ner"), grouped.ner.apply(pd.Series)], axis=1
	)
	l.append(grouped)
df = pd.concat(l)

In [10]:
df["word"] = df.apply(
	lambda row: row["text"][row["start"] : row["end"]], axis=2
)

In [15]:
df_sorted = df.sort_values("word")

In [20]:
df["word"] = df["word"].str.lower()

In [23]:
df.sort_values("text").iloc[0,0]

'"64-111 K 2D pass reads (compared with 21 K for the typical R7.3 experiment)".'

## Check most common entities per category

In [2]:
data = "1500"
def merge_entities(entity_list):
    merged_list = []
    skip = False

    for i in range(len(entity_list)):
        if skip:
            skip = False
            continue

        current_entity = entity_list[i]
        current_entity.pop("word", None)  # Remove 'word' entry
        if current_entity["entity_group"] == "B":
            scores = [current_entity["score"]]
            # Look ahead to find contiguous 'I' entities
            next_idx = i + 1
            while (
                next_idx < len(entity_list)
                and entity_list[next_idx]["entity_group"] == "I"
                and (entity_list[next_idx]["start"] - current_entity["end"]) <= 4
            ):
                scores.append(entity_list[next_idx]["score"])
                current_entity["end"] = entity_list[next_idx]["end"]  # Update end value
                skip = True
                next_idx += 1

            # Average the scores
            current_entity["score"] = sum(scores) / len(scores)

        merged_list.append(current_entity)

    return merged_list
path = f"/home/gomez/gomez/preds{data}/NER_output"

In [3]:
d = pd.read_parquet(f"{path}/COMPOUND.parquet").explode("ner").dropna()

In [4]:
grouped = d.groupby("text").agg({"ner": lambda x: list(x)}).reset_index()
grouped["ner"] = grouped["ner"].apply(merge_entities)
grouped = grouped.explode("ner")
grouped = pd.concat(
                [grouped.drop(columns="ner"), grouped.ner.apply(pd.Series)], axis=1
            )

In [5]:
grouped["word"] = grouped.apply(
            lambda row: row["text"][row["start"] : row["end"]], axis=1
        )

In [6]:
grouped.value_counts("word").head(300).reset_index().word.values

array(['CO_2', '2', 'NaCl', 'H_2O_2', 'cisplatin', 'DMSO', 'IPTG',
       '% CO_2', 'PBS', 'Cd', 'ciprofloxacin', 'EDTA', 'colistin',
       'doxorubicin', 'H_', 'HEPES', '4', 'tetracycline', 'N', 'DOX', 'P',
       'copper', 'CaCl_2', 'O_2', '5-FU', 'gentamicin', 'TMZ', 'HCl',
       'CO', 'Tris', '3', 'phosphate', 'Fe', '5', 'Cu', 'CO_',
       'paclitaxel', 'Ca', 'Zn', 'ampicillin', 'vancomycin',
       'chloroquine', 'KH_2PO_4', 'Dox', 'H_2', 'iron', 'KCl',
       'doxycycline', 'SDS', 'N_2', 'H', 'Al', 'MgCl_2',
       'paraformaldehyde', 'MgSO_4', '2O', 'CQ', '2O_2', 'oxygen', 'H_2O',
       'hydrogen peroxide', 'nitrate', 'K', 'fluconazole', 'meropenem',
       'Na', 'metal', 'zinc', 'gefitinib', 'PTX', 'MeOH', 'rifampicin',
       'PI', 'CDDP', 'PEG', 'Ca^2+', 'Ni', 'curcumin', 'gemcitabine',
       'tigecycline', 'Mn', 'DSS', 'ethanol', 'nitrogen', 'Pb',
       'ceftazidime', 'DTT', 'CO2', 'phenol', 'imipenem', '6', 'MgCl_',
       'cadmium', 'bile', 'C', 'Tris-HCl', '2+', 'ca

In [7]:
word_list = ['2', 'CO_', 'H_', "UV", "% CO_2","3", "6", "1","7","D",'β', 'MgCl_','H_2O_',"NH_","II)", "-"]
sample_row = grouped[grouped['word'].isin(word_list)].groupby("word").apply(lambda df: df.sample(1))




/tmp/ipykernel_19331/1100839408.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_row = grouped[grouped['word'].isin(word_list)].groupby("word").apply(lambda df: df.sample(1))


In [8]:
with open("sentences.txt", "w") as file:
	for sentence in sample_row["text"].values:
		file.write(sentence + "\n")

---

In [9]:
d = pd.read_parquet(f"{path}/ISOLATE.parquet").explode("ner").dropna()
grouped = d.groupby("text").agg({"ner": lambda x: list(x)}).reset_index()
grouped["ner"] = grouped["ner"].apply(merge_entities)
grouped = grouped.explode("ner")
grouped = pd.concat(
                [grouped.drop(columns="ner"), grouped.ner.apply(pd.Series)], axis=1
            )
grouped["word"] = grouped.apply(
            lambda row: row["text"][row["start"] : row["end"]], axis=1
        )

In [10]:
grouped.value_counts("word").head(200).reset_index().word.values

array(['soil', 'blood', 'soil sample', 'rhizosphere', 'soil samples',
       'seawater', 'patient', 'water', 'mangrove', 'kimchi', 'sediment',
       'milk', 'cheese', 'rhizosphere soil', 'urine', 'marine sediment',
       'hot spring', 'freshwater', 'human feces', 'wastewater', 'roots',
       'sewage', 'fecal', 'feces', 'food', 'marine', 'deep-sea', 'leaves',
       'human', 'deep-sea sediment', 'sputum', 'skin', 'wine', 'root',
       'marine sediments', 'gut', 'sourdough', 'stool', 'soils',
       'fecal samples', 'environmental', 'leaf', 'sediments',
       'activated sludge', 'dairy', 'rumen', 'tissue', 'forest soil',
       'meat', 'raw milk', 'hot springs', 'Sea', 'compost', 'human gut',
       'of', 'blood culture', 'surface', 'pig', 'nasal', 'surface water',
       'lymph node', 'human patient', 'human blood', 'rhizospheric soil',
       'patients', 'water samples', 'chicken', 'vineyard', 'chicken meat',
       'spleen', 'river water', 'wound', 'water sample', 'deep-',
      

In [11]:
word_list = ["-","deep-", "lymph node metastasis","bovine mastitis","of", "deep", "old"]
sample_row = grouped[grouped['word'].isin(word_list)].groupby("word").apply(lambda df: df.sample(1))


/tmp/ipykernel_19331/2439090487.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_row = grouped[grouped['word'].isin(word_list)].groupby("word").apply(lambda df: df.sample(1))


In [12]:
with open("sentences.txt", "w") as file:
	for sentence in sample_row["text"].values:
		file.write(sentence + "\n")

---

In [13]:
d = pd.read_parquet(f"{path}/SPECIES.parquet").explode("ner").dropna()
grouped = d.groupby("text").agg({"ner": lambda x: list(x)}).reset_index()
grouped["ner"] = grouped["ner"].apply(merge_entities)
grouped = grouped.explode("ner")
grouped = pd.concat(
                [grouped.drop(columns="ner"), grouped.ner.apply(pd.Series)], axis=1
            )
grouped["word"] = grouped.apply(
            lambda row: row["text"][row["start"] : row["end"]], axis=1
        )

In [14]:
grouped.value_counts("word").head(180).reset_index().word.values

array(['E. coli', 'S. aureus', 'P. aeruginosa', 'Escherichia coli', '.',
       'coli', 'K. pneumoniae', 'S', 'Staphylococcus aureus', 'S.', 'P',
       'B. subtilis', 'L. monocytogenes', 'A. baumannii', 'A',
       'C. albicans', 'Pseudomonas aeruginosa', 'B', 'C. difficile',
       'H. pylori', 'C. jejuni', 'V. cholerae', 'E. faecalis', 'sp.', 'E',
       'C', 'M. tuberculosis', 'Bacillus subtilis', 'S. pneumoniae',
       'B. cereus', 'P.', 'L. plantarum', 'A.', 'Typhimurium', 'sp',
       'S. epidermidis', 'S. cerevisiae', 'S. mutans', 'S. flexneri',
       'S. enterica', 'Salmonella enterica serovar Typhimurium',
       'P. falciparum', 'A. fumigatus', 'B.', 'M. smegmatis',
       'L. pneumophila', 'C.', 'P. gingivalis', 'Klebsiella pneumoniae',
       'L', 'H', 'R', 'Listeria monocytogenes', 'V. parahaemolyticus',
       'B. pseudomallei', 'E. faecium', 'C. perfringens', 'P. putida',
       'Candida albicans', 'M', 'S. typhimurium', 'M.', 'P. fluorescens',
       'aureus', 'F. ox

In [15]:
word_list = ["spp.", "coli", "S","S.","P","C","M","E.","E","B","T","V","N","H","R","B","B.","C.","sp"]
sample_row = grouped[grouped['word'].isin(word_list)].groupby("word").apply(lambda df: df.sample(1))

/tmp/ipykernel_19331/2332379159.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_row = grouped[grouped['word'].isin(word_list)].groupby("word").apply(lambda df: df.sample(1))


In [16]:
with open("sentences.txt", "w") as file:
	for sentence in sample_row["text"].values:
		file.write(sentence + "\n")

---

In [21]:
d = pd.read_parquet(f"{path}/METABOLITE.parquet").explode("ner").dropna()
grouped = d.groupby("text").agg({"ner": lambda x: list(x)}).reset_index()
grouped["ner"] = grouped["ner"].apply(merge_entities)
grouped = grouped.explode("ner")
grouped = pd.concat(
                [grouped.drop(columns="ner"), grouped.ner.apply(pd.Series)], axis=1
            )
grouped["word"] = grouped.apply(
            lambda row: row["text"][row["start"] : row["end"]], axis=1
        )

In [22]:
grouped.value_counts("word").head(280).reset_index().word.values

array(['streptomycin', 'penicillin', 'glucose', 'LPS', 'glycerol',
       'ampicillin', 'EPS', 'ethanol', 'glutamine', 'sucrose',
       'kanamycin', 'L-glutamine', 'xylose', 'acetate', 'amino', 'IAA',
       'chloramphenicol', 'pyruvate', 'gentamicin', 'starch', 'ATP', 'L',
       'lactate', '2', 'ABA', 'arabinose', 'galactose', 'proline', 'SA',
       'fructose', 'acetic acid', 'lactose', 'erythromycin',
       'tetracycline', 'lactic acid', 'cellulose', '-', 'tryptophan',
       'lysine', 'vancomycin', '3', 'glutamate', 'PHB', 'butyrate',
       'arginine', '5', 'lipopolysaccharide', 'Streptomycin',
       'cholesterol', 'Penicillin', 'GABA', 'l-glutamine', 'GA', 'β',
       'siderophore', 'lignin', 'fatty', 'AHL', 'insulin', 'glycine',
       'maltose', 'leucine', 'chlorophyll', 'surfactin', 'JA',
       'succinate', 'rifampicin', 'D', 'PHA', 'trehalose', 'colistin',
       'nisin', 'sodium pyruvate', 'mannose', 'DON', 'sugar',
       'bacteriocin', 'EGF', '4', 'curcumin', 'cellobi

In [23]:
word_list = ["-","2","5","L","3","β","6","fatty","1","α","L-","C","B","D-","3-","poly","n","O"]
sample_row = grouped[grouped['word'].isin(word_list)].groupby("word").apply(lambda df: df.sample(1))

/tmp/ipykernel_19331/2523062150.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_row = grouped[grouped['word'].isin(word_list)].groupby("word").apply(lambda df: df.sample(1))


In [24]:
with open("sentences.txt", "w") as file:
	for sentence in sample_row["text"].values:
		file.write(sentence + "\n")

---

In [25]:
d = pd.read_parquet(f"{path}/ORGANISM.parquet").explode("ner").dropna()
grouped = d.groupby("text").agg({"ner": lambda x: list(x)}).reset_index()
grouped["ner"] = grouped["ner"].apply(merge_entities)
grouped = grouped.explode("ner")
grouped = pd.concat(
                [grouped.drop(columns="ner"), grouped.ner.apply(pd.Series)], axis=1
            )
grouped["word"] = grouped.apply(
            lambda row: row["text"][row["start"] : row["end"]], axis=1
        )

In [26]:
grouped.value_counts("word").head(330).reset_index().word.values

array(['mice', 'MDA-MB-231', 'MCF-7', 'MDA', 'A549', 'HCT116', 'S', 'T',
       'mouse', 'human', 'C', 'P', 'MCF', 'MCF7', 'NCI', 'RAW 264.7',
       'Vero', 'rice', 'HeLa', 'SH-SY5Y', 'A', 'HepG2', 'C. elegans',
       'Caco-2', 'wheat', 'Huh7', 'HCT', 'tomato', 'Caco', 'MB', 'maize',
       'Mice', 'H1299', 'HEK293T', 'HT-29', 'HT29', 'pigs', 'A2780',
       'S. cerevisiae', 'B', 'nude mice', 'F', 'T47D', 'U87', 'chickens',
       'HT', 'U251', 'Arabidopsis', 'THP-1', 'BALB/c mice', 'CCL', 'A375',
       'SW480', 'CRL', 'soybean', 'RAW264.7', 'Col-0', 'cattle', 'PC-3',
       'L', 'SK', '293T', 'C. albicans', 'HEK', 'C57BL/6 mice', 'MDA-MB-',
       'MB-231', 'MDA-MB-468', 'Vero E6', 'HEK293', 'PC', 'PC3', 'CHO',
       'Col', 'larvae', 'chicken', 'worms', 'LNCaP', 'D', 'yeast', 'U',
       'A. thaliana', 'SKOV3', 'THP', 'M', 'MDCK', 'tobacco', 'U2OS',
       'WT', 'humans', 'N', 'HTR-8/SVneo', 'HL', 'piglets', 'murine',
       'H1975', 'breast', 'rat', 'flies', 'rats', 'CHO-K1', 'MC

In [27]:
word_list = ["N","S","A","6","7","T","nude","2","O","G","4","293","5","3","M","H"]
sample_row = grouped[grouped['word'].isin(word_list)].groupby("word").apply(lambda df: df.sample(1))

/tmp/ipykernel_19331/4235904294.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_row = grouped[grouped['word'].isin(word_list)].groupby("word").apply(lambda df: df.sample(1))


In [28]:
with open("sentences.txt", "w") as file:
	for sentence in sample_row["text"].values:
		file.write(sentence + "\n")

---

In [29]:
d = pd.read_parquet(f"{path}/MEDIUM.parquet").explode("ner").dropna()
grouped = d.groupby("text").agg({"ner": lambda x: list(x)}).reset_index()
grouped["ner"] = grouped["ner"].apply(merge_entities)
grouped = grouped.explode("ner")
grouped = pd.concat(
                [grouped.drop(columns="ner"), grouped.ner.apply(pd.Series)], axis=1
            )
grouped["word"] = grouped.apply(
            lambda row: row["text"][row["start"] : row["end"]], axis=1
        )

In [30]:
grouped.value_counts("word").head(190).reset_index().word.values

array(['DMEM', 'LB', 'PBS', 'RPMI 1640', 'RPMI-1640', 'RPMI',
       'Dulbecco’s modified Eagle’s medium', 'TSB', 'LB broth', 'MEM',
       'PDA', 'Luria-Bertani', 'YPD', 'LB agar', 'NGM', 'BHI', 'M9',
       'serum', 'FBS', 'Dulbecco’s Modified Eagle Medium', 'MRS',
       "Dulbecco's modified Eagle's medium", 'DMEM/F12',
       'phosphate-buffered saline', 'Dulbecco’s Modified Eagle’s Medium',
       'serum-free', 'RPMI1640', 'fetal bovine serum', 'EMEM',
       'Dulbecco’s modified Eagle medium', 'TSA', 'MRS broth', 'F12',
       'MS', 'agar', 'potato dextrose agar', 'SD', 'minimal',
       'bovine serum', 'IMDM', 'minimal medium',
       'Roswell Park Memorial Institute', 'F-12', 'glucose',
       'tryptic soy broth', 'lysogeny broth', 'MHB', 'PDB', 'LB medium',
       'BHI broth', 'nutrient broth', 'McCoy’s 5A', 'CDM', 'Tris-HCl',
       'HBSS', 'NB', 'phosphate buffered saline', 'BG11', 'TB', '7H9',
       'α-MEM', '10% FBS', 'LB plates', 'M9 minimal', 'MM',
       'Luria Bertani

In [31]:
word_list = ["Luria-Bertani (","serum","LB)","Dulbecco’s modified Eagle’s medium (DMEM", "LB) broth","the","basal", "LB-","DMEM/","SD/","f/2"]
sample_row = grouped[grouped['word'].isin(word_list)].groupby("word").apply(lambda df: df.sample(1))

/tmp/ipykernel_19331/1973350780.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_row = grouped[grouped['word'].isin(word_list)].groupby("word").apply(lambda df: df.sample(1))


In [32]:
with open("sentences.txt", "w") as file:
	for sentence in sample_row["text"].values:
		file.write(sentence + "\n")

---

In [33]:
d = pd.read_parquet(f"{path}/PHENOTYPE.parquet").explode("ner").dropna()
grouped = d.groupby("text").agg({"ner": lambda x: list(x)}).reset_index()
grouped["ner"] = grouped["ner"].apply(merge_entities)
grouped = grouped.explode("ner")
grouped = pd.concat(
                [grouped.drop(columns="ner"), grouped.ner.apply(pd.Series)], axis=1
            )
grouped["word"] = grouped.apply(
            lambda row: row["text"][row["start"] : row["end"]], axis=1
        )

In [34]:
grouped.value_counts("word").head(200).reset_index().word.values

array(['Gram-negative', 'Gram-positive', 'pathogenic', 'probiotic',
       'non-pathogenic', 'non', 'endophytic', 'hypervirulent', 'virulent',
       'thermophilic', 'gram-negative', 'anaerobic', 'aerobic',
       'gram-positive', 'motile', 'aerobically', 'multidrug',
       'mesophilic', 'anaerobically', 'nonpathogenic', 'toxigenic',
       'halophilic', 'non-motile', 'methicillin', 'nitrogen-fixing',
       'auxotrophic', 'highly pathogenic', 'uropathogenic',
       'Gram negative', 'Gram positive', 'rod', 'Gram', 'nitrogen',
       'white', 'non-', 'rod-shaped', 'halotolerant', 'green',
       'phytopathogenic', 'prototrophic', 'yellow', 'MDR',
       'multidrug-resistant', 'negative', 'nitrogen fixation',
       'psychrophilic', 'drug', 'non-toxigenic', 'thermostable',
       'avirulent', 'thermotolerant', 'heterotrophic', 'autotrophic',
       'mucoid', 'free-living', 'salt', 'double-stranded', 'lytic',
       'cold', 'resistant', 'positive', 'smooth', 'unicellular',
       'diazo

In [35]:
word_list = ["non","non-","N","methicillin","Gram","Gram-","multidrug","self","cold"]
sample_row = grouped[grouped['word'].isin(word_list)].groupby("word").apply(lambda df: df.sample(1))

/tmp/ipykernel_19331/1676419202.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_row = grouped[grouped['word'].isin(word_list)].groupby("word").apply(lambda df: df.sample(1))


In [36]:
with open("sentences.txt", "w") as file:
	for sentence in sample_row["text"].values:
		file.write(sentence + "\n")

---

In [37]:
d = pd.read_parquet(f"{path}/MORPHOLOGY.parquet").explode("ner").dropna()
grouped = d.groupby("text").agg({"ner": lambda x: list(x)}).reset_index()
grouped["ner"] = grouped["ner"].apply(merge_entities)
grouped = grouped.explode("ner")
grouped = pd.concat(
                [grouped.drop(columns="ner"), grouped.ner.apply(pd.Series)], axis=1
            )
grouped["word"] = grouped.apply(
            lambda row: row["text"][row["start"] : row["end"]], axis=1
        )

In [38]:
grouped.value_counts("word").head(210).reset_index().word.values

array(['biofilm', 'biofilms', 'spores', 'spore', 'conidia', 'hyphae',
       'colonies', 'planktonic', 'spheroids', 'hyphal', 'plaques',
       'colony', 'Biofilm', 'aggregates', 'mycelial', 'sporulation',
       'filamentous', 'filaments', 'flagella', 'mycelia', 'rod',
       'mycelium', 'conidial', 'microcolonies', 'Biofilms', 'Spores',
       'pili', 'ascospores', 'pellicle', 'zoospores', 'cysts',
       'vegetative', 'aerial hyphae', 'fruiting bodies', 'endospore',
       'spheres', 'aerial mycelium', 'sphere', 'rods', 'fimbriae',
       'sessile', 'pilus', 'filament', 'Spore', 'spheroid', 'Conidia',
       'plaque', 'fruiting', 'fibrils', 'heterocysts', 'conidiophores',
       'appressoria', 'endospores', 'sporangia', 'hypha', 'microcolony',
       'pseudohyphae', 'appressorium', 'blastospores', 'germ tubes',
       'chlamydospores', 'spore germination', 'conidiation', 'clumps',
       'aerial mycelia', 'coccoid', 'sporulating', 'swarming', 'spindle',
       'pellicles', 'yeast', 

In [39]:
word_list = ["non", "spore-","sporulation"]
sample_row = grouped[grouped['word'].isin(word_list)].groupby("word").apply(lambda df: df.sample(1))

/tmp/ipykernel_19331/1222242915.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_row = grouped[grouped['word'].isin(word_list)].groupby("word").apply(lambda df: df.sample(1))


In [40]:
with open("sentences.txt", "w") as file:
	for sentence in sample_row["text"].values:
		file.write(sentence + "\n")

---

In [41]:
d = pd.read_parquet(f"{path}/EFFECT.parquet").explode("ner").dropna()
grouped = d.groupby("text").agg({"ner": lambda x: list(x)}).reset_index()
grouped["ner"] = grouped["ner"].apply(merge_entities)
grouped = grouped.explode("ner")
grouped = pd.concat(
                [grouped.drop(columns="ner"), grouped.ner.apply(pd.Series)], axis=1
            )
grouped["word"] = grouped.apply(
            lambda row: row["text"][row["start"] : row["end"]], axis=1
        )

In [42]:
grouped.value_counts("word").head(190).reset_index().word.values

array(['growth', 'apoptosis', '-', 'formation', 'of', 'proliferation',
       'antimicrobial', 'stress', 'expression', 'inhibitory',
       'cell death', 'viability', 'cytotoxicity', 'resistance',
       'production', 'antibacterial', 'cytotoxic', 'survival',
       'infection', 'cells', 'tolerance', 'accumulation', 'antifungal',
       'expression of', 'cell viability', 'invasion',
       'cell proliferation', 'degradation', 'protection', 'development',
       'oxidative stress', 'toxicity', 'uptake', 'biocontrol', 'activity',
       'gene expression', 'virulence', 'metabolism', 'migration',
       'replication', 'death', 'antiviral', 'drought stress', 'damage',
       'yield', 'protective', 'tumor growth', 'colonization',
       'plant growth', 'mortality', 'differentiation', 'antioxidant',
       'biosynthesis', 'anti', 'bactericidal', 'germination',
       'cell growth', 'inhibition', 'autophagy', 'sensitivity',
       'salt stress', 'activation', 'release', 'pathogenicity',
      

In [43]:
word_list = ["cell","cells","effect","effects","-","cycle","activity","cell cycle","gene","acid","genes","IL-","challenge","anti","membrane","IFN-", "the", ")"]
sample_row = grouped[grouped['word'].isin(word_list)].groupby("word").apply(lambda df: df.sample(1))

/tmp/ipykernel_19331/3709673889.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_row = grouped[grouped['word'].isin(word_list)].groupby("word").apply(lambda df: df.sample(1))


In [44]:
with open("sentences.txt", "w") as file:
	for sentence in sample_row["text"].values:
		file.write(sentence + "\n")

---

In [45]:
d = pd.read_parquet(f"{path}/PROTEIN.parquet").explode("ner").dropna()
grouped = d.groupby("text").agg({"ner": lambda x: list(x)}).reset_index()
grouped["ner"] = grouped["ner"].apply(merge_entities)
grouped = grouped.explode("ner")
grouped = pd.concat(
                [grouped.drop(columns="ner"), grouped.ner.apply(pd.Series)], axis=1
            )
grouped["word"] = grouped.apply(
            lambda row: row["text"][row["start"] : row["end"]], axis=1
        )

In [46]:
grouped.value_counts("word").head(310).reset_index().word.values

array(['IL', 'GFP', 'anti', 'His', 'TNF', 'GST', 'PE', 'IFN', 'HA',
       'EGFR', 'S', 'protease', 'p53', 'NF', 'CD4', 'β', 'ACE2',
       'mCherry', 'spike', 'SOD', 'APC', 'α', 'CD3', 'N', 'E-cadherin',
       '1', 'AKT', 'IL-1β', 'β-galactosidase', 'Akt', 'IL-', 'NF-κB',
       'RBD', 'CAT', 'β-catenin', 'FITC', 'PD-L1', 'cellulase', 'FLAG',
       'STAT3', 'TLR4', 'HER2', 'EGFP', 'CD8', 'catalase', 'TGF', 'E',
       'xylanase', '2', 'CD44', 'caspase-3', 'ERK', 'Env', 'MBP', 'Cas9',
       'luciferase', 'HLA', 'CD45', 'NS1', 'c', 'laccase', 'chitinase',
       'β-glucosidase', 'IL-6', 'CD11b', 'YFP', 'PI3K', 'NP', 'IgG',
       'VEGF', 'TLR2', '6', 'NA', 'gp120', 'PrP', 'GAPDH', 'HRP', 'JNK',
       'Bcl-2', 'lipase', 'p21', 'C', 'Bax', 'RFP', 'p', 'AR', 'P-gp',
       'vimentin', 'POD', 'Fc', 'ERK1/2', 'S1', 'β-', 'CXCR4', 'LC3',
       'eGFP', 'M', 'PTEN', 'trypsin', 'KPC', 'Flag', '3', 'p38', 'LDH',
       'CD19', 'E2', 'H', 'mTOR', 'iNOS', 'β-actin', 'TNF-α', 'α-amylase',
     

In [47]:
word_list = ["M","C","6","N","1","H","β-","3","5","pro","I","P","p","2","E","B","-"]
sample_row = grouped[grouped['word'].isin(word_list)].groupby("word").apply(lambda df: df.sample(1))

/tmp/ipykernel_19331/1758371248.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_row = grouped[grouped['word'].isin(word_list)].groupby("word").apply(lambda df: df.sample(1))


In [48]:
with open("sentences.txt", "w") as file:
	for sentence in sample_row["text"].values:
		file.write(sentence + "\n")

---

In [50]:
d = pd.read_parquet(f"{path}/DISEASE.parquet").explode("ner").dropna()
grouped = d.groupby("text").agg({"ner": lambda x: list(x)}).reset_index()
grouped["ner"] = grouped["ner"].apply(merge_entities)
grouped = grouped.explode("ner")
grouped = pd.concat(
                [grouped.drop(columns="ner"), grouped.ner.apply(pd.Series)], axis=1
            )
grouped["word"] = grouped.apply(
            lambda row: row["text"][row["start"] : row["end"]], axis=1
        )

In [52]:
grouped.value_counts("word").head(180).reset_index().word.values

array(['breast cancer', 'melanoma', 'cancer', 'COVID', 'colon cancer',
       'HCC', 'lung cancer', 'tumor', 'CRC', 'ovarian cancer', 'NSCLC',
       'glioblastoma', 'glioma', 'prostate cancer', 'neuroblastoma',
       'TNBC', 'human breast cancer', 'gastric cancer', 'GBM',
       'pancreatic cancer', 'colorectal cancer', 'osteosarcoma',
       'outbreak', 'colitis', 'GC', 'hepatoma', 'leukemia',
       'hepatocellular carcinoma', 'breast', 'bladder cancer',
       'human melanoma', 'AML', 'diarrhea', 'cervical cancer',
       'lung adenocarcinoma', '19', 'colon carcinoma', 'MM', 'BC', 'NPC',
       'human colon cancer', 'human cancer', 'COVID-19', 'PDAC',
       'liver cancer', 'HUS', 'powdery mildew', 'AD', 'obesity', 'HNSCC',
       'OSCC', 'Human breast cancer', 'human glioblastoma', 'lung',
       'influenza', 'breast adenocarcinoma', 'ESCC', 'lymphoma', 'colon',
       'PCa', 'pneumonia', 'meningitis', 'lung carcinoma', 'SARS',
       'cholera', 'human lung cancer', 'human neurob

In [53]:
word_list = ["19","19 infection","colon","disease"]
sample_row = grouped[grouped['word'].isin(word_list)].groupby("word").apply(lambda df: df.sample(1))

/tmp/ipykernel_19331/3734180366.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_row = grouped[grouped['word'].isin(word_list)].groupby("word").apply(lambda df: df.sample(1))


In [54]:
with open("sentences.txt", "w") as file:
	for sentence in sample_row["text"].values:
		file.write(sentence + "\n")

## Check strains

In [55]:
d = pd.read_parquet(f"{path}/STRAIN/strains.parquet")

In [62]:
strain_counts = d.value_counts("word")

In [63]:
words = ["antarcticum", "aquificae", "arthrobacter", "1", "-","7","2","sh","3","n2", "coli","a","vero","sars-cov-2","-2","bl21 (de3","hek293t",". coli","atcc","/"]

In [64]:
sample_row = d[d['word'].isin(words)].groupby("word").apply(lambda df: df.sample(1))

/tmp/ipykernel_19331/3058888984.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_row = d[d['word'].isin(words)].groupby("word").apply(lambda df: df.sample(1))


In [65]:
with open("sentences.txt", "w") as file:
	for sentence in sample_row["text"].values:
		file.write(sentence + "\n")